In [252]:
import json
from dataclasses import dataclass
from pathlib import Path
import numpy
from loguru import logger
import pandas as pd
from io import StringIO

In [253]:
anchor_types = {                
                "0": "Interaction",
                "1": "Robot Response",
                "2": "Verbal",
                "3": "gaze/face",
                "4": "gaze/arm",
                "5": "gaze/camera"}


user_info='''021601	Social	task first
021701	Functional	no task first
022401	Social	no task first
022402	Social	task first
022801	Functional	no task first
030301	Functional	task first
030401	Social	no task first
030901	Social	task first
031001	Functional	no task first
031101	Functional	task first
031401	Social	no task first
031501	Social	task first
031701	Functional	no task first
032301	Functional	task first
032801	Social	no task first
032901	Social	task first
032902	Functional	no task first
032903	Functional	task first
032904	Social	no task first
033001	Social	task first
033101	Functional	no task first
040101	Functional	task first
040601	Social	no task first
040801	Social	task first
040802	Functional	no task first
041301	Functional	task first
041501	Social	no task first
042501	Social	task first
050901	Social	no task first
051601	Functional	task first'''
user_info = StringIO(user_info)

users = pd.read_csv(user_info, sep='\t', header=None, names=['uid', 'embodiment', 'task'], dtype=str).set_index('uid')

def parse_video(data: dict, task_first:bool=True):
    # print(users.loc['021601'].task)
    fid = data["fid"]
    fname = data["fname"].split(".")[0]
    split_char = "_" if "_" in fname else "-"
    task = 'undefined' # T/I
    cam = 'cam1'
    for name in fname.split(split_char):
        if 'task' in name:
            task_id = int(name[-1])
            # if task_first:
            #     task = 'T' if task_id == 1 else 'I'
            # else:
            #     task = 'I' if task_id == 1 else 'T'
        elif 'cam' in name:
            cam = name
        elif 'gaze' in name or 'bag' in name:
            cam = 'gaze'
        else:
            if name=='1':
                task_id = 1
                cam = 'cam1'
            elif name=='2':
                task_id = 2
                cam = 'cam1'
            elif name=='3':
                task_id = 1
                cam = 'cam2'
            elif name=='4':
                task_id = 2
                cam = 'cam2'


            # if task_first:
            #     task = 'T' if task_id == 1 else 'I'
            # else:
            #     task = 'I' if task_id == 1 else 'T'

    # logger.info(f"fid:{fid}, cam:{cam}, task:{task}")
    return fid, cam, task_id



In [266]:
path = "/Volumes/Elements/data_annotated"
path = Path(path)
df = pd.DataFrame(columns=['user','cam','task','type','start','end'])
#iterate subfolders
for folder in path.iterdir():
    if not folder.is_dir():
        continue
    uid = folder.name
    # file = folder / (uid + ".json")
    try:
        file = next(folder.glob("*.json"))
        with open(file, "rb") as fp:
            data = json.load(fp)
    except Exception as err:
        logger.error(f"{uid}, {err}")
    # try:
    #     assert data['project']['pname']==uid
    # except Exception as err:
    #     logger.error(f"{uid}, {data['project']['pname']}")
    #     logger.error(err)
    # video sources
    videos = {}
    for _, item in data["file"].items():
        fid, cam, task = parse_video(item, task_first=(users.loc[uid].task=='task first'))
        videos[fid] = {'cam': cam, 'task': task}
    for entry in data['metadata'].values():
        if len(entry['z'])!=2:
            continue
        try:
            row = {'user': uid, 'cam': videos[entry['vid']]['cam'], 'task': videos[entry['vid']]['task'], 'type': anchor_types[entry['av']['1']], 'start': entry['z'][0], 'end': entry['z'][1]}
        except Exception as err:
            logger.error(f"{uid}, {entry['vid']}, {entry['z']}")
            logger.error(err)
        row = pd.DataFrame.from_records([row])
        df = pd.concat([df,row], ignore_index=True)
        
df['duration'] = df.end - df.start
df.cam=df.cam.astype('category')
df.task=df.task.astype('category')
df.type=df.type.astype('category')

2022-06-02 15:39:07.104 | ERROR    | __main__:<cell line: 5>:15 - 040801, 
2022-06-02 15:39:07.136 | ERROR    | __main__:<cell line: 5>:15 - 040802, 
2022-06-02 15:39:07.169 | ERROR    | __main__:<cell line: 5>:15 - 041301, 
2022-06-02 15:39:07.203 | ERROR    | __main__:<cell line: 5>:15 - 041501, 
2022-06-02 15:39:07.236 | ERROR    | __main__:<cell line: 5>:15 - 042501, 
2022-06-02 15:39:07.273 | ERROR    | __main__:<cell line: 5>:15 - 050901, 
2022-06-02 15:39:07.308 | ERROR    | __main__:<cell line: 5>:15 - 051601, 


In [267]:
dt = df.groupby(['user','task','cam','type']).duration.sum()
avg = dt.groupby(['user','task','type']).mean()
avg = avg.unstack(level=2)
avg.to_csv('avg_duration.csv')
avg

type         Interaction  Robot Response     Verbal  gaze/arm  gaze/camera  \
user   task                                                                  
021601 1       26.548983        6.146067  10.586297  0.000000      0.00000   
       2       21.641283        5.208163   9.852580  0.000000      0.00000   
021701 1       37.697467       17.292303  16.879247  0.000000      0.00000   
       2       29.211140       13.596500  11.570730  3.156667      0.00000   
022401 1       36.627390       13.446257   8.636447  0.000000      0.00000   
       2       42.003660       15.909157   7.949773  3.417740      0.00000   
022402 1       24.214743        5.705253   5.101327  0.000000      0.00000   
       2       33.488657        8.442830   8.380383  0.000000      0.00000   
022801 1       20.216023        6.912770   5.885860  0.000000      0.00000   
       2       24.804823        7.726950   6.260540  0.000000      0.00000   
030301 1       28.929990        8.934607   7.542607  3.134043      0.00000   
       2       22.420363        7.009180   7.509320  2.121167      0.00000   
030401 1       33.984347        9.500467   8.857380  0.000000      0.00000   
       2       23.709747        5.881150   4.758837  0.000000      0.00000   
030901 1       26.669040        7.579817   6.238600  0.000000      0.00000   
       2       10.149697        3.124747   3.579217  0.000000      0.00000   
031001 1       26.558533        7.565570  10.119950  0.000000      0.00000   
       2       30.281397        7.852460  10.548520  0.706827      0.00000   
031101 1       30.836707        9.352767   8.236413  0.000000      0.00000   
       2       21.806433        6.875960   8.245240  0.000000      0.00000   
031401 1       27.464913        8.670573   7.169020  0.000000      0.00000   
       2       39.097013       10.522560   8.116957  0.000000      0.00000   
031501 1       44.335570       11.330960  12.646690  0.000000      0.00000   
       2       37.864333       10.497337  10.813883  0.000000      0.00000   
031701 1       48.333023        6.948433  11.404910  0.000000      0.00000   
       2       25.617257        4.646723   6.516470  0.000000      0.00000   
032301 1       25.715727        4.840807  11.677403  0.000000      0.00000   
       2       34.583990        7.668153  17.238350  0.000000      9.86947   
032801 1       33.423487        9.515983  14.322237  0.000000      0.00000   
       2       35.842380        9.206243  16.046077  1.664083      0.00000   
032901 1       40.411900       11.783630  11.268037  0.459197      0.00000   
       2       31.132340        8.975960   9.296993  0.617250      0.00000   
032902 1       25.985900        6.532257   6.767367  0.000000      0.00000   
       2       36.242353        7.913063   8.498670  3.486890      0.00000   
032903 1       32.792237        7.590077   9.958427  2.097773      0.00000   
       2       26.113040        7.353750   7.870350  0.000000      0.00000   
032904 1       32.480800        8.596273   8.571850  0.000000      0.00000   
       2       30.908017        8.777537   8.871207  0.198090      0.00000   
033001 1       11.971170        3.276210   5.822780  0.000000      0.00000   
       2       12.271640        6.081393   2.782527  0.666667      0.00000   
033101 1       35.939123        8.986437   9.783753  0.000000      0.00000   
       2       35.655453        8.709427   9.730873  0.000000      0.00000   
040101 1       29.280437        7.049623   8.109363  1.610570      0.00000   
       2       23.902643        6.645410   7.271337  0.000000      0.00000   
040601 1       23.520140        6.022647   5.898893  0.000000      0.00000   
       2       31.668080        8.122610   7.707523  0.000000      0.00000   
040801 1       23.520140        6.022647   5.898893  0.000000      0.00000   
       2       31.668080        8.122610   7.707523  0.000000      0.00000   
040802 1       23.520140        6.022647   5.898893  0.000000      0.00000   
       2       31.6680

In [268]:
group = df.groupby(['user','task'])
session = group.end.max() - group.start.min()
session.rename("whole", inplace=True)
session = session.to_frame()

output = pd.merge(avg, session, left_index=True, right_index=True)
output.to_csv('avg_duration.csv')

In [269]:
output

Interaction  Robot Response     Verbal  gaze/arm  gaze/camera  \
user   task                                                                  
021601 1       26.548983        6.146067  10.586297  0.000000      0.00000   
       2       21.641283        5.208163   9.852580  0.000000      0.00000   
021701 1       37.697467       17.292303  16.879247  0.000000      0.00000   
       2       29.211140       13.596500  11.570730  3.156667      0.00000   
022401 1       36.627390       13.446257   8.636447  0.000000      0.00000   
       2       42.003660       15.909157   7.949773  3.417740      0.00000   
022402 1       24.214743        5.705253   5.101327  0.000000      0.00000   
       2       33.488657        8.442830   8.380383  0.000000      0.00000   
022801 1       20.216023        6.912770   5.885860  0.000000      0.00000   
       2       24.804823        7.726950   6.260540  0.000000      0.00000   
030301 1       28.929990        8.934607   7.542607  3.134043      0.00000   
       2       22.420363        7.009180   7.509320  2.121167      0.00000   
030401 1       33.984347        9.500467   8.857380  0.000000      0.00000   
       2       23.709747        5.881150   4.758837  0.000000      0.00000   
030901 1       26.669040        7.579817   6.238600  0.000000      0.00000   
       2       10.149697        3.124747   3.579217  0.000000      0.00000   
031001 1       26.558533        7.565570  10.119950  0.000000      0.00000   
       2       30.281397        7.852460  10.548520  0.706827      0.00000   
031101 1       30.836707        9.352767   8.236413  0.000000      0.00000   
       2       21.806433        6.875960   8.245240  0.000000      0.00000   
031401 1       27.464913        8.670573   7.169020  0.000000      0.00000   
       2       39.097013       10.522560   8.116957  0.000000      0.00000   
031501 1       44.335570       11.330960  12.646690  0.000000      0.00000   
       2       37.864333       10.497337  10.813883  0.000000      0.00000   
031701 1       48.333023        6.948433  11.404910  0.000000      0.00000   
       2       25.617257        4.646723   6.516470  0.000000      0.00000   
032301 1       25.715727        4.840807  11.677403  0.000000      0.00000   
       2       34.583990        7.668153  17.238350  0.000000      9.86947   
032801 1       33.423487        9.515983  14.322237  0.000000      0.00000   
       2       35.842380        9.206243  16.046077  1.664083      0.00000   
032901 1       40.411900       11.783630  11.268037  0.459197      0.00000   
       2       31.132340        8.975960   9.296993  0.617250      0.00000   
032902 1       25.985900        6.532257   6.767367  0.000000      0.00000   
       2       36.242353        7.913063   8.498670  3.486890      0.00000   
032903 1       32.792237        7.590077   9.958427  2.097773      0.00000   
       2       26.113040        7.353750   7.870350  0.000000      0.00000   
032904 1       32.480800        8.596273   8.571850  0.000000      0.00000   
       2       30.908017        8.777537   8.871207  0.198090      0.00000   
033001 1       11.971170        3.276210   5.822780  0.000000      0.00000   
       2       12.271640        6.081393   2.782527  0.666667      0.00000   
033101 1       35.939123        8.986437   9.783753  0.000000      0.00000   
       2       35.655453        8.709427   9.730873  0.000000      0.00000   
040101 1       29.280437        7.049623   8.109363  1.610570      0.00000   
       2       23.902643        6.645410   7.271337  0.000000      0.00000   
040601 1       23.520140        6.022647   5.898893  0.000000      0.00000   
       2       31.668080        8.122610   7.707523  0.000000      0.00000   
040801 1       23.520140        6.022647   5.898893  0.000000      0.00000   
       2       31.668080        8.122610   7.707523  0.000000      0.00000   
040802 1       23.520140        6.022647   5.898893  0.000000      0.00000   
       2       31.668080        8.1